In [2]:
import os
import pandas as pd

directory = os.path.expanduser('~/Documents/NUVU/NUVU/Caudal_Medio_Dia_Historico/')
directory_norm = os.path.expanduser('~/Documents/NUVU/NUVU/Caudal_Medio_Dia_Historico_Normalizado/')

estaciones = {}
estaciones_norm = {}

# Función para procesar archivos en un directorio
def process_directory(dir_path, data_dict):
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            path = os.path.join(dir_path, filename)
            var_name = os.path.splitext(filename)[0].replace(' ', '_').lower()
            df = pd.read_csv(path, sep=';', encoding='ISO-8859-1')
            data_dict[var_name] = df
            print(f"{var_name}")


# Procesar ambos directorios
process_directory(directory, estaciones)
process_directory(directory_norm, estaciones_norm)

# Función para transformar datos
def transform_data(data, lat, lon, station_name):
    melted_data = data.melt(id_vars=["year", "month"], var_name="day", value_name="Caudal")
    melted_data.dropna(subset=["Caudal"], inplace=True)
    melted_data['day'] = melted_data['day'].astype(int)
    melted_data['Fecha'] = pd.to_datetime(melted_data[['year', 'month', 'day']]) + pd.to_timedelta('00:00:00')
    melted_data.sort_values('Fecha', inplace=True)
    melted_data['Latitud'] = lat
    melted_data['Longitud'] = lon
    melted_data['Estacion'] = station_name.replace('_', ' ').title()
    return melted_data

# Coordenadas de cada estación
coords = {
    "la_esperanza_nechi": (8.030111111, -74.78555556),
    "la_raya": (8.344555556, -74.56122222),
    "las_varas": (8.39, -74.56),
    "magangue": (9.24, -74.74166667),
    "sucre": (8.809722222, -74.72222222),
    "tres_cruces": (8.703083333, -74.51730556),
    "villa_teresa": (9.023972222, -74.97666667)
}

# Transformar datos y añadir columna 'Caudal_norm'
transformed_data = {}
for name, df in estaciones.items():
    lat, lon = coords[name]
    station_name = name  # Mantén el nombre original para obtener el nombre de la estación
    transformed_df = transform_data(df, lat, lon, station_name)
    transformed_data[name] = transformed_df

for name, norm_df in estaciones_norm.items():
    base_name = name.replace('_norm', '')
    if base_name in transformed_data:
        station_name = base_name  # Utiliza el nombre base para los datos normalizados
        norm_df_melted = transform_data(norm_df, None, None, station_name)
        norm_df_melted = norm_df_melted[['Fecha', 'Caudal']].rename(columns={'Caudal': 'Caudal_norm'})
        transformed_data[base_name] = pd.merge(transformed_data[base_name], norm_df_melted, on='Fecha', how='left')


# Imprimir los primeros registros de los DataFrames transformados para verificar
for name, df in transformed_data.items():
    print(f"--- {name} ---")
    print(df.head())


la_esperanza_nechi
la_raya
las_varas
magangue
sucre
tres_cruces
villa_teresa
las_varas_norm
la_esperanza_nechi_norm
la_raya_norm
magangue_norm
sucre_norm
tres_cruces_norm
villa_teresa_norm
--- la_esperanza_nechi ---
   year  month  day Caudal      Fecha   Latitud   Longitud  \
0  1966      9    1   1350 1966-09-01  8.030111 -74.785556   
1  1966      9    2   1312 1966-09-02  8.030111 -74.785556   
2  1966      9    3   1235 1966-09-03  8.030111 -74.785556   
3  1966      9    4   1155 1966-09-04  8.030111 -74.785556   
4  1966      9    5   1175 1966-09-05  8.030111 -74.785556   

             Estacion  Caudal_norm  
0  La Esperanza Nechi     0.591766  
1  La Esperanza Nechi     0.557505  
2  La Esperanza Nechi     0.534569  
3  La Esperanza Nechi     0.549945  
4  La Esperanza Nechi     0.573743  
--- la_raya ---
   year  month  day Caudal      Fecha   Latitud   Longitud Estacion  \
0  1975      6    1  304,7 1975-06-01  8.344556 -74.561222  La Raya   
1  1975      6    2  309,3 1975

In [3]:
for name, df in transformed_data.items():
    df['Caudal'] = df['Caudal'].astype(str).str.replace(',', '.').astype(float)
    
    if 'Caudal_norm' in df.columns:
        df['Caudal_norm'] = df['Caudal_norm'].astype(str).str.replace(',', '.').astype(float)

for name, df in transformed_data.items():
    print(f"--- {name} ---")
    print(df.head())


--- la_esperanza_nechi ---
   year  month  day  Caudal      Fecha   Latitud   Longitud  \
0  1966      9    1  1350.0 1966-09-01  8.030111 -74.785556   
1  1966      9    2  1312.0 1966-09-02  8.030111 -74.785556   
2  1966      9    3  1235.0 1966-09-03  8.030111 -74.785556   
3  1966      9    4  1155.0 1966-09-04  8.030111 -74.785556   
4  1966      9    5  1175.0 1966-09-05  8.030111 -74.785556   

             Estacion  Caudal_norm  
0  La Esperanza Nechi     0.591766  
1  La Esperanza Nechi     0.557505  
2  La Esperanza Nechi     0.534569  
3  La Esperanza Nechi     0.549945  
4  La Esperanza Nechi     0.573743  
--- la_raya ---
   year  month  day  Caudal      Fecha   Latitud   Longitud Estacion  \
0  1975      6    1   304.7 1975-06-01  8.344556 -74.561222  La Raya   
1  1975      6    2   309.3 1975-06-02  8.344556 -74.561222  La Raya   
2  1975      6    3   333.5 1975-06-03  8.344556 -74.561222  La Raya   
3  1975      6    4   354.5 1975-06-04  8.344556 -74.561222  La Raya

In [4]:
combined_data = pd.concat(transformed_data.values(), ignore_index=True)

In [8]:
properties = ['Fecha', 'Caudal', 'Caudal_norm', 'Estacion']

def df_to_geojson(df, properties, lat='Latitud', lon='Longitud'):
    geojson = {'type': 'FeatureCollection', 'features': []}
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'Point',
                'coordinates': [row[lon], row[lat]]
            }
        }
        for prop in properties:
            if isinstance(row[prop], pd.Timestamp):
                feature['properties'][prop] = row[prop].isoformat()
            else:
                feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [9]:
import json

geojson_data = df_to_geojson(combined_data, properties)

geojson_path = os.path.expanduser('~/Documents/NUVU/NUVU/Caudales_norm_etiquetas.geojson')
with open(geojson_path, 'w') as f:
    json.dump(geojson_data, f)

print(f"GeoJSON file created at {geojson_path}")


GeoJSON file created at C:\Users\nuvup/Documents/NUVU/NUVU/Caudales_norm_etiquetas.geojson
